In [2]:
import cobra
import pandas as pd
import json
from cobra import Model, Reaction, Metabolite
from cobra.io import load_model, write_sbml_model, read_sbml_model, save_json_model
from pathlib import Path
from collections import defaultdict

import networkx as nx
import matplotlib.pyplot as plt

In [3]:
SBMLmodel = Path("iJV803_M_album_BG8_GEM_R_full_version_updated.sbml")
model = read_sbml_model(SBMLmodel)

'' is not a valid SBML 'SId'.


In [4]:
model.optimize

<bound method Model.optimize of <Model  at 0x21dbd87ee40>>

In [ ]:
partial_name = "phosphatidyl"

# Find reactions that contain the partial name
matching_reactions = [rxn for rxn in model.metabolites if partial_name.lower() in rxn.name.lower()]

# Display results
if matching_reactions:
    for rxn in matching_reactions:
        print(f"{rxn.id}")
else:
    print("No reactions found containing that name.")

Diisohexadecanoylphosphatidylserine_c
Diisohexadecanoylphosphatidylglycerophosphate_c
Diisohexadecanoylphosphatidylglycerol_c
Diisopentadecanoylphosphatidylglycerophosphate_c
Diisotetradecanoylphosphatidylserine_c
phosphatidylserine_ditetradecanoyl_c
phosphatidylserine_dihexadec_9_enoyl_c
phosphatidylethanolamine_ditetradecanoyl_c
phosphatidylserine_dioctadecanoyl_c
phosphatidylethanolamine_dioctadecanoyl_c
Dianteisopentadecanoylphosphatidylserine_c
L_2_Lysophosphatidylethanolamine_c
Phosphatidylglycerophosphate_dihexadec_9_enoyl_c
Phosphatidylglycerol_dihexadec_9_enoyl_c
Dianteisopentadecanoylphosphatidylglycerophosphate_c
Phosphatidylglycerophosphate_dihexadecanoyl_c
Phosphatidylglycerophosphate_dioctadec_11_enoyl_c
Diisohexadecanoylphosphatidylethanolamine_c
Dianteisoheptadecanoylphosphatidylserine_c
1_Phosphatidyl_1D_myo_inositol_3_4_bisphosphate_c
1_Phosphatidyl_1D_myo_inositol_3_phosphate_c
phosphatidylserine_didodecanoyl_c
phosphatidylethanolamine_didodecanoyl_c
Dianteisopentade

In [ ]:
met = model.metabolites.get_by_id("fa4coa_c")
met.summary()


In [ ]:
for reaction in model.reactions:
    if "biomass" in reaction.id.lower():  
        print(F"Reaction ID: {reaction.id}")
        print(F"Reaction Name: {reaction.name}")
        print(F"Reaction: {reaction.reaction}")
        print(50 * "-")


In [ ]:
def download_SBML():
    try:
        write_sbml_model(model, 'BG8EnhancedModel.xml')

        with open('model.xml', 'r') as file:
            sbml_content = file.read()

        return sbml_content
        
    except Exception as e:
        raise Exception(f"Error saving or loading SBML model: {str(e)}")
    

download_SBML()


In [5]:
save_json_model(model, "BG8EnhancedModel.json")

In [5]:
solution = model.optimize()
fluxes = solution.fluxes
flux_dict = fluxes.to_dict()
with open("fluxData.json" , "w") as f: 
    json.dump(flux_dict, f)

In [ ]:
solution

In [ ]:
with open("output_reactions.json", "r") as f:
    output_reactions = json.load(f)

unique_metabolites = set()

for rxn_id in output_reactions:
    try:
        rxn = model.reactions.get_by_id(rxn_id)
        
        if rxn.metabolites:
            for metabolite in rxn.metabolites:
                unique_metabolites.add(metabolite.id)

    except KeyError:
        print(f"Reaction {rxn_id} not found in the model")

unique_metabolites_list = sorted(list(unique_metabolites))

with open("unique_metabolites.json", "w") as outfile:
    json.dump(unique_metabolites_list, outfile, indent=4)

print("Unique metabolites written to unique_metabolites.json")

Unique metabolites written to unique_metabolites.json


In [14]:
with open("output_reactions.json", "r") as f:
    output_reactions = json.load(f)

unique_metabolites = set()

for rxn_id in output_reactions:
    try:
        rxn = model.reactions.get_by_id(rxn_id)
        
        if rxn.metabolites:
            for metabolite in rxn.metabolites:
                metabolite_id = metabolite.id
                
                # Filter: ignore metabolites with less than 5 characters or "ose_c"
                if len(metabolite_id) >= 6 and "ose_c" not in metabolite_id:
                    unique_metabolites.add(metabolite_id)

    except KeyError:
        print(f"Reaction {rxn_id} not found in the model")

unique_metabolites_list = sorted(list(unique_metabolites))


with open("filtered_metabolites.json", "w") as outfile:
    json.dump(unique_metabolites_list, outfile, indent=4)

print("Filtered metabolites written to filtered_metabolites.json")


Filtered metabolites written to filtered_metabolites.json


In [ ]:
with open("filtered_metabolites.json") as f:
    filtered_metabolite_ids = json.load(f)

metabolite_list = []
for met_id in filtered_metabolite_ids:
    metabolite = model.metabolites.get_by_id(met_id) 
    metabolite_list.append({
        "id": met_id,
        "name": metabolite.name if metabolite else "Unknown",
        "compartment": metabolite.compartment if metabolite else "Unknown"
    })

df = pd.DataFrame(metabolite_list)

print(df)

# Save as JSON
with open("metabolites_summary.json", "w") as f:
    json.dump(metabolite_list, f, indent=4)

In [ ]:
#ddca_c  = dodecanoate

reactions = [
    {"id": "rxn08803_c", "reactants": ["H2O_c", "1_Acyl_sn_glycero_3_phosphoethanolamine_dodecanoyl_c"], "products": ["H_c", "Glycerophosphoethanolamine_c", "ddca_c"]},
    {"id": "rxn08804_c", "reactants": ["H2O_c", "1_Acyl_sn_glycero_3_phosphoethanolamine_tetradecanoyl_c"], "products": ["H_c", "Glycerophosphoethanolamine_c", "Myristic_acid_c"]},
    {"id": "rxn08806_c", "reactants": ["H2O_c", "1_Acyl_sn_glycero_3_phosphoethanolamine_hexadecanoyl_c"], "products": ["H_c", "Glycerophosphoethanolamine_c", "Palmitate_c"]},
    {"id": "rxn08808_c", "reactants": ["H2O_c", "H_c", "L_2_Lysophosphatidylethanolamine_c"], "products": ["H_c", "Glycerophosphoethanolamine_c", "ocdca_c"]},
    {"id": "rxn08805_c", "reactants": ["H2O_c", "1_Acyl_sn_glycero_3_phosphoethanolamine_tetradec_7_enoyl_c"], "products": ["H_c", "Glycerophosphoethanolamine_c", "tetradecenoate_c"]},
    {"id": "rxn08807_c", "reactants": ["H2O_c", "2_Acyl_sn_glycero_3_phosphoethanolamine_hexadec_9_enoyl_c"], "products": ["H_c", "Glycerophosphoethanolamine_c", "hexadecenoate_c"]},
    {"id": "rxn08809_c", "reactants": ["H2O_c", "L_2_Lysophosphatidylethanolamine_c"], "products": ["H_c", "Glycerophosphoethanolamine_c", "octadecenoate_c"]},
    {"id": "rxn08838_c", "reactants": ["H2O_c", "2_Acyl_sn_glycero_3_phosphoethanolamine_dodecanoyl_c"], "products": ["H_c", "Glycerophosphoethanolamine_c", "ddca_c"]},
    {"id": "rxn08839_c", "reactants": ["H2O_c", "2_Acyl_sn_glycero_3_phosphoethanolamine_tetradecanoyl_c"], "products": ["H_c", "Glycerophosphoethanolamine_c", "Myristic_acid_c"]},
    {"id": "rxn08841_c", "reactants": ["H2O_c", "2_Acyl_sn_glycero_3_phosphoethanolamine_n_C16_0_c"], "products": ["H_c", "Glycerophosphoethanolamine_c", "Palmitate_c"]},
    {"id": "rxn08843_c", "reactants": ["H2O_c", "2_Acyl_sn_glycero_3_phosphoethanolamine_n_C18_0_c"], "products": ["H_c", "Glycerophosphoethanolamine_c", "ocdca_c"]},
    {"id": "rxn08840_c", "reactants": ["H2O_c", "2_Acyl_sn_glycero_3_phosphoethanolamine_tetradec_7_enoyl_c"], "products": ["H_c", "Glycerophosphoethanolamine_c", "tetradecenoate_c"]},
    {"id": "rxn08842_c", "reactants": ["H2O_c", "1_Acyl_sn_glycero_3_phosphoethanolamine_hexadec_9_enoyl_c"], "products": ["H_c", "Glycerophosphoethanolamine_c", "hexadecenoate_c"]},
    {"id": "rxn08844_c", "reactants": ["H2O_c", "2_Acyl_sn_glycero_3_phosphoethanolamine_octadec_11_enoyl_c"], "products": ["H_c", "Glycerophosphoethanolamine_c", "octadecenoate_c"]},
    {"id": "rxn08796_c", "reactants": ["H2O_c", "1_dodecanoyl_sn_glycerol_3_phosphate_c"], "products": ["H_c", "Glycerol_3_phosphate_c", "ddca_c"]},
    {"id": "rxn08797_c", "reactants": ["H2O_c", "1_tetradecanoyl_sn_glycerol_3_phosphate_c"], "products": ["H_c", "Glycerol_3_phosphate_c", "Myristic_acid_c"]},
    {"id": "rxn08799_c", "reactants": ["H2O_c", "1_hexadecanoyl_sn_glycerol_3_phosphate_c"], "products": ["H_c", "Glycerol_3_phosphate_c", "Palmitate_c"]},
    {"id": "rxn08801_c", "reactants": ["H2O_c", "1_octadec_11_enoyl_sn_glycerol_3_phosphate_c"], "products": ["H_c", "Glycerol_3_phosphate_c", "ocdca_c"]},
    {"id": "rxn08798_c", "reactants": ["H2O_c", "1_tetradec_7_enoyl_sn_glycerol_3_phosphate_c"], "products": ["H_c", "Glycerol_3_phosphate_c", "tetradecenoate_c"]},
    {"id": "rxn08800_c", "reactants": ["H2O_c", "1_hexadec_9_enoyl_sn_glycerol_3_phosphate_c"], "products": ["H_c", "hexadecenoate_c", "Glycerol_3_phosphate_c"]},
    {"id": "rxn08802_c", "reactants": ["H2O_c", "1_octadec_11_enoyl_sn_glycerol_3_phosphate_c"], "products": ["H_c", "Glycerol_3_phosphate_c", "octadecenoate_c"]},
    {"id": "rxn08817_c", "reactants": ["H2O_c", "2_dodecanoyl_sn_glycerol_3_phosphate_c"], "products": ["H_c", "Glycerol_3_phosphate_c", "ddca_c"]},
    {"id": "rxn08818_c", "reactants": ["H2O_c", "2_tetradecanoyl_sn_glycerol_3_phosphate_c"], "products": ["H_c", "Glycerol_3_phosphate_c", "Myristic_acid_c"]},
    {"id": "rxn08820_c", "reactants": ["H2O_c", "2_hexadecanoyl_sn_glycerol_3_phosphate_c"], "products": ["H_c", "Glycerol_3_phosphate_c", "Palmitate_c"]},
    {"id": "rxn08822_c", "reactants": ["H2O_c", "2_octadecanoyl_sn_glycerol_3_phosphate_c"], "products": ["H_c", "Glycerol_3_phosphate_c", "ocdca_c"]},
    {"id": "rxn08819_c", "reactants": ["H2O_c", "2_tetradec_7_enoyl_sn_glycerol_3_phosphate_c"], "products": ["H_c", "Glycerol_3_phosphate_c", "tetradecenoate_c"]},
    {"id": "rxn08821_c", "reactants": ["H2O_c", "2_hexadec_9_enoyl_sn_glycerol_3_phosphate_c"], "products": ["H_c", "Glycerol_3_phosphate_c", "hexadecenoate_c"]},
    {"id": "rxn08823_c", "reactants": ["H2O_c", "2_octadec_11_enoyl_sn_glycerol_3_phosphate_c"], "products": ["H_c", "Glycerol_3_phosphate_c", "octadecenoate_c"]},
    {"id": "rxn08810_c", "reactants": ["H2O_c", "1_Acyl_sn_glycero_3_phosphoglycerol_dodecanoyl_c"], "products": ["H_c", "ddca_c", "Glycerophosphoglycerol_c"]},
    {"id": "rxn08811_c", "reactants": ["H2O_c", "1_Acyl_sn_glycero_3_phosphoglycerol_tetradecanoyl_c"], "products": ["H_c", "Glycerophosphoglycerol_c", "Myristic_acid_c"]},
    {"id": "rxn08813_c", "reactants": ["H2O_c", "1_Acyl_sn_glycero_3_phosphoglycerol_hexadecanoyl_c"], "products": ["H_c", "Glycerophosphoglycerol_c", "Palmitate_c"]},
    {"id": "rxn08815_c", "reactants": ["H2O_c", "1_Acyl_sn_glycero_3_phosphoglycerol_octadecanoyl_c"], "products": ["H_c", "Glycerophosphoglycerol_c", "ocdca_c"]},
    {"id": "rxn08812_c", "reactants": ["H2O_c", "1_Acyl_sn_glycero_3_phosphoglycerol_tetradec_7_enoyl_c"], "products": ["H_c", "Glycerophosphoglycerol_c", "tetradecenoate_c"]},
    {"id": "rxn08814_c", "reactants": ["H2O_c", "1_Acyl_sn_glycero_3_phosphoglycerol_hexadec_9_enoyl_c"], "products": ["H_c", "hexadecenoate_c", "Glycerophosphoglycerol_c"]},
    {"id": "rxn08816_c", "reactants": ["H2O_c", "1_Acyl_sn_glycero_3_phosphoglycerol_octadec_11_enoyl_c"], "products": ["H_c", "octadecenoate_c", "Glycerophosphoglycerol_c"]},
    {"id": "rxn08845_c", "reactants": ["H2O_c", "2_Acyl_sn_glycero_3_phosphoglycerol_dodecanoyl_c"], "products": ["H_c", "ddca_c", "Glycerophosphoglycerol_c"]},
    {"id": "rxn08846_c", "reactants": ["H2O_c", "2_Acyl_sn_glycero_3_phosphoglycerol_tetradecanoyl_c"], "products": ["H_c", "Glycerophosphoglycerol_c", "Myristic_acid_c"]},
    {"id": "rxn08848_c", "reactants": ["H2O_c", "2_Acyl_sn_glycero_3_phosphoglycerol_hexadecanoyl_c"], "products": ["H_c", "Glycerophosphoglycerol_c", "Palmitate_c"]},
    {"id": "rxn08850_c", "reactants": ["H2O_c", "2_Acyl_sn_glycero_3_phosphoglycerol_octadecanoyl_c"], "products": ["H_c", "Glycerophosphoglycerol_c", "ocdca_c"]},
    {"id": "rxn08847_c", "reactants": ["H2O_c", "2_Acyl_sn_glycero_3_phosphoglycerol_tetradec_7_enoyl_c"], "products": ["H_c", "Glycerophosphoglycerol_c", "tetradecenoate_c"]},
    {"id": "rxn08849_c", "reactants": ["H2O_c", "2_Acyl_sn_glycero_3_phosphoglycerol_hexadec_9_enoyl_c"], "products": ["H_c", "Glycerophosphoglycerol_c", "hexadecenoate_c"]},
    {"id": "rxn08851_c", "reactants": ["H2O_c", "2_Acyl_sn_glycero_3_phosphoglycerol_octadec_11_enoyl_c"], "products": ["H_c", "Glycerophosphoglycerol_c", "octadecenoate_c"]},
]

In [81]:
met = model.metabolites.get_by_id("1_Acyl_sn_glycero_3_phosphoethanolamine_dodecanoyl_c")
met

Metabolite identifier,1_Acyl_sn_glycero_3_phosphoethanolamine_dodecanoyl_c
Name,1-Acyl-sn-glycero-3-phosphoethanolamine-...
Memory address,0x197df98b470
Formula,C17H36NO7P
Compartment,c
In 1 reaction(s),rxn08803_c


In [78]:
rxn = model.reactions.get_by_id("rxn08803_c")
rxn

Reaction identifier,rxn08803_c
Name,"Lysophospholipase L1 (2-acylglycerophosphoethanolamine, n-C12:0) (periplasm)"
Memory address,0x197dfeb5d30
Stoichiometry,1_Acyl_sn_glycero_3_phosphoethanolamine_dodecanoyl_c + H2O_c <=> Glycerophosphoethanolamine_c + H_c + ddca_c 1-Acyl-sn-glycero-3-phosphoethanolamine-dodecanoyl-c + H2O-e <=> Glycerophosphoethanolamine-c + H-e + ddca-c
GPR,Malb_1519 or Malb_14
Lower bound,-1000.0
Upper bound,1000.0


In [ ]:
G = nx.DiGraph()

reactions = [
    ("1-Acyl-PE (dodecanoyl)", "Glycerophosphoethanolamine")
    ("1-Acyl-PE (dodecanoyl)", "Dodecanoate (ddca)")
    ("1-Acyl-PE (tetradecanoyl)", "Glycerophosphoethanolamine")
    ("1-Acyl-PE (tetradecanoyl)", "Myristic acid")
    ("1-Acyl-PE (hexadecanoyl)", "Glycerophosphoethanolamine")
    ("1-Acyl-PE (hexadecanoyl)", "Palmitate")
    ("Lysophosphatidylethanolamine", "Glycerophosphoethanolamine")
    ("Lysophosphatidylethanolamine", "ocdca")
    ("1-Acyl-PE (tetradec_7_enoyl)", "Glycerophosphoethanolamine")
    ("1-Acyl-PE (tetradec_7_enoyl)", "Tetradecenoate")

    ("2-Acyl-PE (dodecanoyl)", "Glycerophosphoethanolamine")
    ("2-Acyl-PE (dodecanoyl)", "Dodecanoate (ddca)")
    ("2-Acyl-PE (tetradecanoyl)", "Glycerophosphoethanolamine")
    ("2-Acyl-PE (tetradecanoyl)", "Myristic acid")
    ("2-Acyl-PE (hexadecanoyl)", "Glycerophosphoethanolamine")
    ("2-Acyl-PE (hexadecanoyl)", "Palmitate")
    
    ("2-Acyl-PE (octadec_11_enoyl)", "Glycerophosphoethanolamine")
    ("2-Acyl-PE (octadec_11_enoyl)", "Octadecenoate")
    ("1-Acyl-G3P (dodecanoyl)", "Glycerophosphoglycerol")
    ("1-Acyl-G3P (dodecanoyl)", "Dodecanoate (ddca)")
    ("1-Acyl-G3P (tetradecanoyl)", "Glycerophosphoglycerol")
    ("1-Acyl-G3P (tetradecanoyl)", "Myristic acid")
    ("1-Acyl-G3P (hexadecanoyl)", "Glycerophosphoglycerol")
    ("1-Acyl-G3P (hexadecanoyl)", "Palmitate")
    ("1-Acyl-G3P (octadecanoyl)", "Glycerophosphoglycerol")
    ("1-Acyl-G3P (octadecanoyl)", "ocdca")
    ("1-Acyl-G3P (tetradec_7_enoyl)", "Glycerophosphoglycerol")
    ("1-Acyl-G3P (tetradec_7_enoyl)", "Tetradecenoate")
    ("2-Acyl-G3P (dodecanoyl)", "Glycerophosphoglycerol")
    ("2-Acyl-G3P (dodecanoyl)", "Dodecanoate (ddca)")
    ("2-Acyl-G3P (tetradecanoyl)", "Glycerophosphoglycerol")
    ("2-Acyl-G3P (tetradecanoyl)", "Myristic acid")
    ("2-Acyl-G3P (hexadecanoyl)", "Glycerophosphoglycerol")
    ("2-Acyl-G3P (hexadecanoyl)", "Palmitate")
    ("2-Acyl-G3P (octadecanoyl)", "Glycerophosphoglycerol")
    ("2-Acyl-G3P (octadecanoyl)", "ocdca")
    ("2-Acyl-G3P (tetradec_7_enoyl)", "Glycerophosphoglycerol")
    ("2-Acyl-G3P (tetradec_7_enoyl)", "Tetradecenoate")
    ("2-Acyl-G3P (hexadec_9_enoyl)", "Glycerophosphoglycerol")
    ("2-Acyl-G3P (hexadec_9_enoyl)", "Hexadecenoate")
    ("2-Acyl-G3P (octadec_11_enoyl)", "Glycerophosphoglycerol")
    ("2-Acyl-G3P (octadec_11_enoyl)", "Octadecenoate")
]

# Add nodes and edges
for reactant, product in reactions:
    G.add_edge(reactant, product)

# Define categories
fatty_acids = {"Dodecanoate", "Myristic acid", "Tetradecenoate", "Palmitate", "Octadecenoate"}
backbones = {"Glycerophosphoethanolamine", "Glycerophosphoglycerol", "Glycerol-3-phosphate"}

# Assign colors
node_colors = []
for node in G.nodes():
    if node in fatty_acids:
        node_colors.append("lightcoral")  # Fatty acids = Red
    elif node in backbones:
        node_colors.append("lightgreen")  # Backbones = Green
    else:
        node_colors.append("lightblue")  # Others = Blue

# Draw graph
plt.figure(figsize=(100, 60))
pos = nx.spring_layout(G, seed=42, k=1.5)
nx.draw(G, pos, with_labels=True, node_size=20000, node_color=node_colors, edge_color="gray", font_size=40, font_weight="bold")
plt.title("Phospholipid Deacylation Pathway")
plt.show()


In [12]:
partial_names = [
    "dodec", "tetra", "tridec", "penta", "hexa", "octa", "hepta", "methyl",
    "propyl", "decanoyl", "palmi", "doco", "erucic", "icosen", "lauroyl",
    "myrist", "nervon", "propionyl", "tiglyl", "lipid", "fa11", "fa12",
    "fa1", "fa3", "fa4", "fa6", "ethanolamine", "ddca", "ocdca",
    "phosphoglycerol", "phosphotidate", "phosphotidyl"
]

excluded_metabolites = {
    "tetrahydrofolate",
    "5_10_Methylenetetrahydrofolate",
    "cyclohexanone",
    "dimethylbenzimidazole",
    "5_Methylcytosine",
    "S_Aminomethyldihydrolipoylprotein",
    "propionyl_phosphate",
    "hexanesulfonate",
    "6_Hydroxymethyl_dihydropterin"
}

matching_metabolites = {
    met.id: {
        "id": met.id,
        "name": met.name,
        "compartment": met.compartment
    }
    for met in model.metabolites
    if any(partial.lower() in met.name.lower() for partial in partial_names)  
    and not any(excluded.lower() in met.name.lower() for excluded in excluded_metabolites)  
}

with open("whole_model_filtered_metabolites.json", "w") as f:
    json.dump(list(matching_metabolites.keys()), f, indent=4)

with open("whole_model_metabolites_summary.json", "w") as f:
    json.dump(list(matching_metabolites.values()), f, indent=4)

df = pd.DataFrame(matching_metabolites.values())
print(df)


                                                    id  \
0    2_Amino_4_hydroxy_6_hydroxymethyl_7_8_dihydrop...   
1                                                fa6_c   
2                                             fa6coa_c   
3                     7_methyl_trans_oct_2_enoyl_ACP_c   
4                              7_methyl_octanoyl_ACP_c   
..                                                 ...   
439                     core_oligosaccharide_lipid_A_c   
440                           Lauroyl_KDO2_lipid_IVA_c   
441                                 Palmitoleic_acid_c   
442                    cpd16330_PQQ_dicarboxylicAcid_c   
443            5_10_Methylenetetrahydromethanopterin_c   

                                                  name compartment  
0    2-Amino-4-hydroxy-6-hydroxymethyl-7-8-dihydrop...           c  
1                                                fa6-c           c  
2                                             fa6coa-c           c  
3                     7-met

In [9]:
with open("filtered_metabolites.json") as f:
    filtered_metabolites = set(json.load(f)) 

with open("whole_model_filtered_metabolites.json") as f:
    whole_model_filtered_metabolites = set(json.load(f))  

count_filtered = len(filtered_metabolites)
count_whole_model_filtered = len(whole_model_filtered_metabolites)

in_filtered_not_in_model = filtered_metabolites - whole_model_filtered_metabolites
in_model_not_in_filtered = whole_model_filtered_metabolites - filtered_metabolites

print(f"Number of metabolites in filtered_metabolites.json: {count_filtered}")
print(f"Number of metabolites in whole_model_filtered_metabolites.json: {count_whole_model_filtered}\n")

print("Metabolites in filtered but NOT in whole model filtered:")
print("--------------------------------------------------------")
if in_filtered_not_in_model:
    print("\n".join(in_filtered_not_in_model))
else:
    print("None")
print() 

print("Metabolites in whole model filtered but NOT in filtered:")
print("--------------------------------------------------------")
if in_model_not_in_filtered:
    print("\n".join(in_model_not_in_filtered))
else:
    print("None")
print()  



Number of metabolites in filtered_metabolites.json: 194
Number of metabolites in whole_model_filtered_metabolites.json: 455

Metabolites in filtered but NOT in whole model filtered:
--------------------------------------------------------
Isovaleryl_CoA_c
2_Oxoglutarate_c
Lignoceroyl_CoA_c
Stearoylcardiolipin_B_subtilis_c
Malonyl_CoA_c
CMP_KDO_c

Metabolites in whole model filtered but NOT in filtered:
--------------------------------------------------------
Isoheptadecanoyllipoteichoic_acid_n24__linked__N_acetyl_D_glucosamine_c
6_Oxo_2_hydroxycyclohexane_1_carboxyl_CoA_c
5_Methyltetrahydropteroyltri_L_glutamate_c
3_Methylsalicylate_c
1_2_ditetradecanoyl_sn_glycerol_3_phosphate_c
7Z_10Z_13Z_16Z_Docosatetraenoyl_CoA_c
S_Hydroxyhexanoyl_CoA_c
2_Octaprenyl_3_methyl_6_methoxy_1_4_benzoquinone_c
3alpha_7alpha_12alpha_24_Tetrahydroxy_5beta_cholestanoyl_CoA_c
Lipid_IVA_c
Isohexadecanoyllipoteichoic_acid_n24__linked__N_acetyl_D_glucosamine_c
2_Demethylmenaquinol_8_c
cis_2_Methyl_5_isopropylhex